# Quality check for each cleaned dataset

## Import packages 

In [1]:
import pandas as pd
import numpy as np
from openpyxl import load_workbook
import itertools

## sheet names

In [2]:
f = load_workbook("data_cleaned/carbon_bombs_all_datasets.xlsx")
sheet_names = f.sheetnames

In [3]:
sheet_names

['carbon_bombs_informations',
 'metadatas',
 'company_informations',
 'bank_informations',
 'connexion_bank_company',
 'connexion_carbonbombs_company',
 'country_informations']

Load all datasets

In [17]:
bank_df = pd.read_excel("data_cleaned/carbon_bombs_all_datasets.xlsx", sheet_name="bank_informations")
cb_df = pd.read_excel("data_cleaned/carbon_bombs_all_datasets.xlsx", sheet_name="carbon_bombs_informations")
comp_df = pd.read_excel("data_cleaned/carbon_bombs_all_datasets.xlsx", sheet_name="company_informations")
cnx_bank_comp_df = pd.read_excel("data_cleaned/carbon_bombs_all_datasets.xlsx", sheet_name="connexion_bank_company")
cnx_cb_comp_df = pd.read_excel("data_cleaned/carbon_bombs_all_datasets.xlsx", sheet_name="connexion_carbonbombs_company")
countries_df = pd.read_excel("data_cleaned/carbon_bombs_all_datasets.xlsx", sheet_name="country_informations")
metadatas = pd.read_excel("data_cleaned/carbon_bombs_all_datasets.xlsx", sheet_name="metadatas")

## Check on bank_informations

- Impossible to check source it's online
- all bank name is in `connexion_bank_company`

In [18]:
bank_df.sample(5, random_state=42)

,Bank Name,Bank Website,Headquarters address,Headquarters country,CEO Name,Board description,Supervisor Name,Supervisor Website,Shareholder structure source,Source BankTrack,Latitude,Longitude,Bank logo,World Region
0,ANZ,http://www.anz.com.au,"833 Collins Street, Docklands,3008 Melbourne",Australia,Shayne Elliott,http://shareholder.anz.com/personnel/shayne-el...,Reserve Bank of Australia,http://www.rba.gov.au/,https://www.marketscreener.com/AUSTRALIA-AND-N...,https://www.banktrack.org/bank/anz,-37.821942,144.945525,https://www.banktrack.org/thumbimage.php?image...,Oceania
5,Bank of America,http://www.bankofamerica.com,"100 North Tryon Street,28255 Charlotte, NC",United States,Brian T. Moynihan,https://about.bankofamerica.com/en-us/who-we-a...,Federal Reserve,http://www.federalreserve.gov/,https://www.nasdaq.com/symbol/bac/ownership-su...,https://www.banktrack.org/bank/bank_of_america,35.227393,-80.842169,https://www.banktrack.org/thumbimage.php?image...,North America
34,MUFG,http://www.bk.mufg.jp/global/,"7-1 Marunouchi 2-Chome Chiyoda,100-0005 Tokyo",Japan,Junichi Hanzawa,http://www.bk.mufg.jp/global/aboutus/managemen...,Bank of Japan,http://www.boj.or.jp/en/,https://www.marketscreener.com/MITSUBISHI-UFJ-...,https://www.banktrack.org/bank/bank_of_tokyo_m...,35.680054,139.763022,https://www.banktrack.org/thumbimage.php?image...,Asia
13,China Construction Bank,http://en.ccb.com/en/home/indexv3.html,"25 Finance Street, Xicheng District,Beijing",China,Tian Guoli,http://en.ccb.com/en/investorv3/biographiesofd...,China Banking and Investment Regulatory Commis...,https://www.cbirc.gov.cn/en/view/pages/index/i...,http://www.huijin-inv.cn/wps/portal/!ut/p/a1/0...,https://www.banktrack.org/bank/china_construct...,39.907110,116.374150,https://www.banktrack.org/thumbimage.php?image...,Asia
44,Rabobank,http://www.rabobank.nl,"Croeselaan 18,3521 CB Utrecht",Netherlands,Stefaan Decraene,https://www.rabobank.nl/over-ons/organisatie/o...,De Nederlandsche Bank,http://www.dnb.nl,https://www.rabobank.com/nl/about-rabobank/coo...,https://www.banktrack.org/bank/rabobank,52.086422,5.109218,https://www.banktrack.org/thumbimage.php?image...,Europe


In [19]:
banks = set(bank_df["Bank Name"])
banks_conx = set(cnx_bank_comp_df["Bank"])

assert len(banks - banks_conx) == 0

## Check on carbon_bombs_information
- len == 425
- Status_source_CB 
    - is "operating" or "not started"
    - match new project "*" in orig file
- Carbon_bomb_name_source_CB 
    - match all CB name in orig file
    - full match cnx cb comp
- Country_source_CB
    - match all CB country in orig file
- Potential_GtCO2_source_CB
    - match all CB column in orig file
- Fuel_type_source_CB
    - match all CB column in orig file
- GEM_id_source_GEM :
    - Values in unit id in mine or gasoil source OR no information
    - show number of no information
- GEM_url_source_GEM :
    - Values in unit url in mine or gasoil source OR no information
    - show number of no information
    - match GEM_id_source_GEM for number of no information
- Latitude, Longitude :
    - no missing value or 0, 0
    - no dupplicated rows
- Latitude_longitude_source :
    - values are GEM or Country CB
    - no missing values
- Operators_source_GEM :
    - Split |
    - merge GEM id and check same operators
    - show amount of "" or None
- Parent_company_source_GEM :
    - nb of "" is same than Operators_source_GEM NAN
    - show amount of "" or None
    - find same companies (TODO)
- Companies_involved_source_GEM :
    - if Parent_company_source_GEM == "" or None in Parent_company_source_GEM then Parent_company_source_GEM else Operators_source_GEM
- Multiple_unit_concerned_source_GEM :
    - not empty only if ; in GEM_id_source_GEM (same number of ; and |)
    - if | in Operators_source_GEM not empty (same number of |)
    - if | in Parent_company_source_GEM not empty (same number of |)
- Status_source_GEM
    - match all CB column in orig file
- Status_lvl_1
- Status_lvl_2

In [20]:
cb_df.isna().mean()

Carbon_bomb_name_source_CB            0.000000
Country_source_CB                     0.000000
World_region                          0.000000
Potential_GtCO2_source_CB             0.000000
Fuel_type_source_CB                   0.000000
GEM_id_source_GEM                     0.000000
GEM_url_source_GEM                    0.000000
Latitude                              0.000000
Longitude                             0.000000
Latitude_longitude_source             0.000000
Operators_source_GEM                  0.214118
Parent_company_source_GEM             0.000000
Companies_involved_source_GEM         0.000000
Multiple_unit_concerned_source_GEM    0.960000
Carbon_bomb_description               0.000000
Carbon_bomb_start_year                0.000000
Status_source_CB                      0.000000
Status_source_GEM                     0.200000
Status_lvl_1                          0.000000
Status_lvl_2                          0.000000
dtype: float64

In [21]:
from carbon_bomb import load_carbon_bomb_coal_database, load_carbon_bomb_gasoil_database

In [22]:
# CB source
cb_gasoil_source_df = pd.read_excel("data_sources/1-s2.0-S0301421522001756-mmc2.xlsx", sheet_name="Oil&Gas", skipfooter = 4,skiprows=1)
cb_gasoil_source_df = cb_gasoil_source_df.loc[:,["New","Project","Country","Gt CO2"]]
cb_gasoil_source_df.columns = ["New","Project Name","Country","Potential emissions (GtCO2)"]
cb_gasoil_source_df["Fuel"] = "Oil&Gas"
cb_gasoil_source_df['Country'] = cb_gasoil_source_df['Country'].replace({
    'Russian Federation': 'Russia',
    'Turkey': 'Türkiye',
    'Saudi-Arabia':'Saudi Arabia',
    'Kuwait-Saudi-Arabia-Neutral Zone':'Kuwait' # see Readme to get the details of this choice
})

cb_coal_source_df = pd.read_excel("data_sources/1-s2.0-S0301421522001756-mmc2.xlsx", sheet_name="Coal", skipfooter = 3)
cb_coal_source_df = cb_coal_source_df.loc[:,["New","Project Name","Country", "Potential emissions (GtCO2)","Fuel"]]
cb_coal_source_df['Country'] = cb_coal_source_df['Country'].replace({'Russian Federation': 'Russia',
                                       'Turkey': 'Türkiye'})

cb_source_df = pd.concat([cb_coal_source_df, cb_gasoil_source_df])
cb_source_df["Project Name"] = cb_source_df["Project Name"].str.strip()

In [23]:
# GEM source
gem_coal_df = pd.read_excel("./data_sources/Global-Coal-Mine-Tracker-April-2023.xlsx", sheet_name='Global Coal Mine Tracker')
gem_gasoil_df = pd.read_excel("data_sources/Global-Oil-and-Gas-Extraction-Tracker-Feb-2023.xlsx", sheet_name='Main data',engine='openpyxl')

C:\Users\natha\apps\miniforge3\envs\carbon_bombs\lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [24]:
# - len == 425
assert len(cb_df) == 425

# merge with CB source
merge = cb_df.merge(
    cb_source_df, left_on=["Carbon_bomb_name_source_CB", "Country_source_CB"], right_on=["Project Name", "Country"], how="inner"
)
assert len(merge) == len(cb_df)

# - Status_source_CB 
#     - is "operating" or "not started"
#     - match new project "*" in orig file
assert cb_df["Status_source_CB"].isna().sum() == 0
assert set(cb_df["Status_source_CB"].unique()) == {"operating", "not started"}
assert all((merge["New"] == "*") == (merge["Status_source_CB"] == "not started"))
assert all((merge["New"] != "*") == (merge["Status_source_CB"] == "operating"))

# - Carbon_bomb_name_source_CB 
#     - match all CB name in orig file
#     - full match cnx cb comp
assert set(cb_df["Carbon_bomb_name_source_CB"]) == set(cb_source_df["Project Name"])
# TODO : check for CB with only Others..
# cb_df.loc[~cb_df["Carbon_bomb_name_source_CB"].isin(cnx_cb_comp_df["Carbon_bomb_name"])]
# assert set(cb_df["Carbon_bomb_name_source_CB"]) == set(cnx_cb_comp_df["Carbon_bomb_name"])

# - Country_source_CB
#     - match all CB country in orig file
assert all((merge["Country"] == merge["Country_source_CB"]))

# - Potential_GtCO2_source_CB
#     - match all CB column in orig file
assert all((merge["Potential emissions (GtCO2)"] == merge["Potential_GtCO2_source_CB"]))

# - Fuel_type_source_CB
#     - match all CB column in orig file
assert all((merge["Fuel"] == merge["Fuel_type_source_CB"]))

# - GEM_id_source_GEM :
#     - Values in unit id in mine or gasoil source OR no information
#     - show number of no information
units = cb_df["GEM_id_source_GEM"].str.split('|')
units = set(itertools.chain(*units))
assert len(
    units - set(gem_coal_df["Mine IDs"]) - set(gem_gasoil_df["Unit ID"]) - {"No informations available on GEM"}
) == 0
print("Nb of 'no information' for GEM_id_source_GEM:", sum(cb_df["GEM_id_source_GEM"] == "No informations available on GEM"))

# - GEM_url_source_GEM :
#     - Values in unit url in mine or gasoil source OR no information
#     - show number of no information
#     - match GEM_id_source_GEM for number of no information
url = cb_df["GEM_url_source_GEM"].str.split('|')
url = set(itertools.chain(*url))
assert len(
    url - set(gem_coal_df["GEM Wiki Page (ENG)"]) - set(gem_gasoil_df["Wiki URL"]) - {"No informations available on GEM"}
) == 0
assert all(
    (
        cb_df["GEM_url_source_GEM"] == "No informations available on GEM"
    ) == (
        cb_df["GEM_id_source_GEM"] == "No informations available on GEM"
    )
)

# - Latitude, Longitude :
#     - no missing value or 0, 0
#     - no dupplicated rows
assert cb_df["Latitude"].isna().sum() == 0
assert cb_df["Longitude"].isna().sum() == 0
assert cb_df[["Latitude", "Longitude"]].duplicated().sum() == 0

# - Latitude_longitude_source :
#     - values are GEM or Country CB
#     - no missing values
assert cb_df["Latitude_longitude_source"].isna().sum() == 0
if not set(cb_df["Latitude_longitude_source"]) == {"GEM", "Country CB"}:
    print(set(cb_df["Latitude_longitude_source"]))

# - Operators_source_GEM :
#     - Split |
#     - merge GEM id and check same operators
#     - show amount of "" or None
a = gem_coal_df[["Mine IDs", "Operators"]]
b = gem_gasoil_df[["Unit ID", "Operator"]]
a.columns = ["ID", "Operator"]
b.columns = ["ID", "Operator"]
gem_id_df = pd.concat([a, b])

def match_gem_id_to_operator(x):
    if x == ["No informations available on GEM"]:
        return ["None"]
    
    res = []
    for unit_id in x:
        ope = gem_id_df.loc[gem_id_df["ID"] == unit_id, "Operator"].fillna("None").values[0]
        res.append(ope)
    
    return res
    
operators = cb_df["Operators_source_GEM"].fillna("None").str.split('|')
assert all(
    operators == cb_df["GEM_id_source_GEM"].str.split('|').apply(match_gem_id_to_operator)
)

# - Parent_company_source_GEM :
#     - show amount of "" or None
#     - find same companies (TODO)

# - Companies_involved_source_GEM :
#     - if Parent_company_source_GEM == "" or None in Parent_company_source_GEM then Parent_company_source_GEM else Operators_source_GEM
# problem 
# assert all(
#     cb_df["Companies_involved_source_GEM"] == np.where(
#         cb_df["Parent_company_source_GEM"] == "No informations on company (100.0%)",
#         cb_df["Operators_source_GEM"],
#         cb_df["Parent_company_source_GEM"]
#     )
# )

# - Multiple_unit_concerned_source_GEM :
#     - not empty only if ; in GEM_id_source_GEM (same number of ; and |)
#     - if | in Operators_source_GEM not empty (same number of |)
#     - if | in Parent_company_source_GEM not empty (same number of |)
# - Status_source_GEM
#     - match all CB column in orig file
# - Status_lvl_1
# - Status_lvl_2


Nb of 'no information' for GEM_id_source_GEM: 83


## Check on company_informations

In [25]:
# bank_df

## Check on connexion_bank_company

In [26]:
assert len(set(cnx_bank_comp_df["Bank"]) - set(bank_df["Bank Name"])) == 0
assert len(set(cnx_bank_comp_df["Company"]) - set(comp_df["Company_name"])) == 0

## Check on connexion_carbonbombs_company

In [27]:
assert len(set(cnx_cb_comp_df["Carbon_bomb_name"]) - set(cb_df["Carbon_bomb_name_source_CB"])) == 0
assert len(set(cnx_cb_comp_df["Company"].fillna("None")) - set(comp_df["Company_name"]) - {"None", 'No informations on company'})  == 0
assert len(set(comp_df["Company_name"]) - set(cnx_cb_comp_df["Company"]) - {"None"})  == 0

## Check on country_informations

In [28]:
# No need UN data is good quality

## Check Metadata

In [29]:
for sheet_name in sheet_names:
    if sheet_name == "metadatas":
        continue

    meta_df = metadatas.loc[metadatas["sheetName"] == sheet_name]
    df_ = pd.read_excel("data_cleaned/carbon_bombs_all_datasets.xlsx", sheet_name=sheet_name)
    
    if set(df_.columns) == set(meta_df["columnName"]):
        print(f"For {sheet_name}: columns OK")
    else:
        print(f"For {sheet_name}: columns NOT OK")
        print(set(df_.columns) - set(meta_df["columnName"]))
        print(set(meta_df["columnName"]) - set(df_.columns))
    print()

For carbon_bombs_informations: columns OK

For company_informations: columns OK

For bank_informations: columns OK

For connexion_bank_company: columns OK

For connexion_carbonbombs_company: columns OK

For country_informations: columns OK

